In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import proplot as pplt
import numpy as np
from rolling_plot_utils import (
    plot_years_and_seasons,
    get_season,
    get_cmap_plus_white,
    BASELINE_SURVEY_START_MJD,
    MJD_2024,
    YEAR,
    get_sun_ra_at_mjd,
    plot_sun_ra,
    plot_rizexptime_fancy,
    mad,
)

In [9]:
import os
import glob

import numpy as np
from rubin_sim import maf
import fitsio
import joblib

from rubin_sim.maf import ExgalM5WithCuts


def _process_file(filename):
    nside = 128

    runname = os.path.basename(filename).replace('.db', '')
    oname = runname + "_exgalm5withcuts_nside%d.fits" % (nside)

    if not os.path.exists(oname):
        print("starting %s" % os.path.basename(filename), flush=True)

        metric = ExgalM5WithCuts()

        slicer = maf.HealpixSlicer(nside=nside, use_cache=False)
        sql = 'note not like "%%DD%%" and night <= 3653 '
        bundle = maf.MetricBundle(metric, slicer, sql, run_name=runname)
        bg = maf.MetricBundleGroup([bundle], filename, '.', None)
        bg.run_all()
        maps_arr = bundle.metric_values.copy().T

        maps_arr = maps_arr.filled(fill_value=-9999)

        fitsio.write(
            oname,
            maps_arr,
            clobber=True,
        )


def main():
    fnames = glob.glob(
        os.path.expandvars(
            os.path.join(
                "${RUBIN_SIM_DATA_DIR}",
                'matts_extra_sims',
                '*.db',
            )
        )
    )
    sims = [
        "roll_uniform_early_half_mjdp0_v3.4_10yrs.db",
        "baseline_v3.4_10yrs.db",
        "noroll_mjdp0_v3.4_10yrs.db",
    ]
    fnames = [
        fname for fname in fnames
        if any(fname.endswith(sim) for sim in sims)
    ]
    jobs = []
    for fname in fnames:
        jobs.append(
            joblib.delayed(_process_file)(fname)
        )


    print("running %d jobs" % len(jobs), flush=True)
    with joblib.Parallel(n_jobs=4, verbose=100, backend="loky") as par:
        par(jobs)

main()

running 3 jobs
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
starting noroll_mjdp0_v3.4_10yrs.db
starting roll_uniform_early_half_mjdp0_v3.4_10yrs.db
starting baseline_v3.4_10yrs.db


OSError: The throughput file /Users/beckermr/RUBIN_SIM_DATA_DIR/throughputs/baseline/total_u.dat does not exist